# Agentic RAG with LlamaIndex


In this notebook we will experiment RAG with multi-document agent.

- Define a reader to read the `pdf` sample file [AraGPT2](./data/aragpt2.pdf) paper.
- Define a `splitter` to process the texts of the document.
- Set the LLM embedding and generation model ids.
- Create the engines from the Indexes and define a tool wrapper around them.
- Create Index for tool objects.
- Define the agent worker and agent runner that utilize memory.
- Excute the multi-docs agent.


## Setups


In [1]:
from rich import print
from dotenv import load_dotenv

In [2]:
# load env variables
_ = load_dotenv()

In [3]:
# define some constants
GENERATION_MODEL_ID = "gpt-4o-mini"
EMBEDDING_MODEL_ID = "text-embedding-3-small"

In [4]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model=GENERATION_MODEL_ID)
Settings.embed_model = OpenAIEmbedding(model=EMBEDDING_MODEL_ID)

## Load Documents


In [5]:
from llama_index.core.schema import TextNode
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter



def get_nodes(file_paths: list[str]) -> dict[str, TextNode]:
    """Extract text nodes from documents.
    
    inputs:
        file_paths (list[str]): paths to pdf files. must be unique.
    returns:
        nodes_dict (dict[str, TextNode]): mapping of file paths to nodes.
    """
    nodes_dict = {file_path.split("/")[-1].split(".")[0]: [] for file_path in file_paths}
    documents_reader = SimpleDirectoryReader(input_files=file_paths)
    documents = documents_reader.load_data()
    sentence_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=64)
    nodes = sentence_splitter.get_nodes_from_documents(documents)
    for node in nodes:
        nodes_dict[node.metadata["file_name"].split(".")[0]].append(node)
    return nodes_dict

In [6]:
import glob

file_paths = glob.glob("data/*")
print(file_paths)

['data/arabert.pdf', 'data/gpt2.pdf', 'data/aragpt2.pdf', 'data/camel_parser.pdf', 'data/camel_bert.pdf']

In [7]:
nodes = get_nodes(file_paths=file_paths)

In [8]:
print(nodes.keys())

dict_keys(['arabert', 'gpt2', 'aragpt2', 'camel_parser', 'camel_bert'])

## Define Vector Search and Summary Tools


In [9]:
from llama_index.core import VectorStoreIndex, SummaryIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata


def get_tools_from_nodes(
    nodes: dict[str, TextNode]
) -> dict[str, list[QueryEngineTool]]:
    """Define query engine tools from nodes dictionary.

    inputs:
        nodes (dict[str, TextNode]): text nodes for each document by name.
    returns:
        tools_dict (dict[str, list[QueryEngineTool]]): tools for each document by name.
    """
    tools_dict = {file_name: [] for file_name in nodes.keys()}
    for file_name, text_nodes in nodes.items():
        print(f"Creating tools for file: {file_name}")
        # define vector tool
        vector_index = VectorStoreIndex(nodes=text_nodes)
        vector_engine = vector_index.as_query_engine()
        vector_metadata = ToolMetadata(
            name=f"vector_tool_for_{file_name}",
            description=f"Useful for retrieving specific context from the {file_name} paper.",
        )
        vector_tool = QueryEngineTool(
            query_engine=vector_engine, metadata=vector_metadata
        )
        # define summary tool
        summary_index = SummaryIndex(nodes=text_nodes)
        summary_engine = summary_index.as_query_engine(response_mode="tree_summarize")
        summary_metadata = ToolMetadata(
            name=f"summary_tool_for_{file_name}",
            description=f"Useful for summarization questions related to the {file_name} paper.",
        )
        summary_tool = QueryEngineTool(
            query_engine=summary_engine, metadata=summary_metadata
        )
        tools_dict[file_name].extend([vector_tool, summary_tool])
    return tools_dict

In [10]:
tools = get_tools_from_nodes(nodes)

Creating tools for file: arabert

Creating tools for file: gpt2

Creating tools for file: aragpt2

Creating tools for file: camel_parser

Creating tools for file: camel_bert

In [11]:
print(tools["arabert"][0])

<llama_index.core.tools.query_engine.QueryEngineTool object at 0x7f5022b1cc50>

## Define Objects Vector Search Tool

This will do vector search over the tools we have.


In [12]:
from llama_index.core.objects import ObjectIndex, ObjectRetriever


def get_object_tool(tools: dict[str, list[QueryEngineTool]]) -> ObjectRetriever:
    """Define tools query engine tool.

    inputs:
        tools (dict[str, list[QueryEngineTool]]): tools for each document by name.
    returns:
        tools_retriever (ObjectRetriever): tools object retriever.
    """
    tools_index = ObjectIndex.from_objects(
        objects=[tl for _, tls in tools.items() for tl in tls], index_cls=VectorStoreIndex
    )
    tools_retriever = tools_index.as_retriever(similarity_top_k=4)
    return tools_retriever

In [13]:
tools_retriever = get_object_tool(tools)
print(tools_retriever)

<llama_index.core.objects.base.ObjectRetriever object at 0x7f5022b200d0>

## Define Agent Worker and Runner


In [14]:
sys_prompt = """\
You are an agent designed to answer queries over a set of \
given papers. Please always use the tools provided to answer \
a question. Do not rely on prior knowledge.\
"""

print(sys_prompt)

You are an agent designed to answer queries over a set of given papers. Please always use the tools provided to 
answer a question. Do not rely on prior knowledge.

In [15]:
from llama_index.core.agent import (
    AgentRunner,
    FunctionCallingAgentWorker,
)

agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=tools_retriever,
    system_prompt=sys_prompt,
    verbose=True,
)

agent = AgentRunner(agent_worker)

In [16]:
query = "Compare between the AraBERT and AraGPT2 models on a high level."


response = agent.chat(query)

Added user message to memory: Compare between the AraBERT and AraGPT2 models on a high level.
=== Calling Function ===
Calling function: vector_tool_for_aragpt2 with args: {"input": "high level comparison between AraBERT and AraGPT2"}
=== Function Output ===
AraBERT and AraGPT2 are both transformer-based models designed for the Arabic language, but they serve different purposes and are built on distinct methodologies. 

AraBERT primarily focuses on natural language understanding (NLU) tasks and is pre-trained using the Masked Language Modeling (MLM) approach. This model is optimized for tasks such as sentence classification and entity recognition, leveraging a masked token prediction strategy to understand context and relationships within the text.

In contrast, AraGPT2 is designed for language generation, specifically causal language modeling. It is trained from scratch on a large corpus of Arabic text, enabling it to generate coherent and contextually relevant text. AraGPT2 comes in 

In [17]:
print(response.response)

AraBERT and AraGPT2 are both transformer-based models tailored for the Arabic language, but they have different 
focuses and methodologies:

### AraBERT:
- **Purpose**: Primarily designed for natural language understanding (NLU) tasks.
- **Training Methodology**: Pre-trained using the Masked Language Modeling (MLM) approach, which involves 
predicting masked tokens in a sentence.
- **Applications**: Optimized for tasks such as sentence classification and named entity recognition, leveraging 
its ability to understand context and relationships within the text.
- **Architecture**: Similar to BERT, it utilizes a bidirectional context understanding, making it effective for 
comprehension tasks.

### AraGPT2:
- **Purpose**: Focused on language generation, specifically causal language modeling.
- **Training Methodology**: Trained from scratch on a large corpus of Arabic text, enabling it to generate coherent
and contextually relevant text.
- **Applications**: Excels in tasks like synthetic news generation, conversational agents, and zero-shot question 
answering. It can produce human-like articles and responses.
- **Architecture**: Built on the GPT-2 architecture, which is optimized for predicting the next word in a sequence 
based on preceding context.

### Summary:
- **AraBERT** is tailored for understanding and processing Arabic text, making it suitable for comprehension tasks.
- **AraGPT2** is optimized for generating text and maintaining context over longer passages, reflecting its 
strengths in creative writing and conversational applications.

Each model serves distinct purposes within Arabic natural language processing, showcasing their respective 
strengths in understanding versus generation.